In [1]:
from db_app.database import engine
import numpy as np
import pandas as pd
import seaborn as sns
import datetime as dt
import matplotlib.pyplot as plt
from catboost import CatBoostClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import roc_auc_score

In [2]:
user_data = pd.read_sql(
                        "SELECT * FROM public.user_data;",
                        con=engine,
                        index_col="user_id"
                        )
post_data = pd.read_sql(
                        "SELECT * FROM public.post_text_df;",
                        con=engine,
                        index_col="post_id"
                        )
feed_data = pd.read_sql(
                        "SELECT * FROM public.feed_data limit 10000;",
                        con=engine
                        )

In [3]:
user_data.head(10)

,gender,age,country,city,exp_group,os,source
user_id,,,,,,,
200,1,34,Russia,Degtyarsk,3,Android,ads
201,0,37,Russia,Abakan,0,Android,ads
202,1,17,Russia,Smolensk,4,Android,ads
203,0,18,Russia,Moscow,1,iOS,ads
204,0,36,Russia,Anzhero-Sudzhensk,3,Android,ads
205,0,32,Russia,Dugulubgey,3,Android,ads
206,1,20,Russia,Kamenka,1,Android,ads
207,1,27,Russia,Groznyy,3,iOS,ads
208,1,17,Russia,Zima,1,Android,ads


In [4]:
post_data.head(10)

,text,topic
post_id,,
1,UK economy facing major risks\n\nThe UK manufa...,business
2,Aids and climate top Davos agenda\n\nClimate c...,business
3,Asian quake hits European shares\n\nShares in ...,business
4,India power shares jump on debut\n\nShares in ...,business
5,Lacroix label bought by US firm\n\nLuxury good...,business
6,Insurance bosses plead guilty\n\nAnother three...,business
14,Saudi investor picks up the Savoy\n\nLondons f...,business
2366,RACV | Annual cost of owning and maintaining a...,covid
7,Turkey-Iran mobile deal at risk\n\nTurkeys inv...,business


In [5]:
feed_data.head(10)

,timestamp,user_id,post_id,action,target
0,2021-10-21 19:37:09,118420,1369,view,0
1,2021-10-21 19:37:59,118420,1093,view,0
2,2021-10-21 19:38:32,118420,2584,view,0
3,2021-10-21 19:39:46,118420,5355,view,0
4,2021-10-21 19:42:16,118420,838,view,0
5,2021-10-21 19:42:31,118420,948,view,0
6,2021-10-21 19:43:20,118420,6927,view,0
7,2021-10-21 19:45:40,118420,1888,view,0
8,2021-10-21 19:48:34,118420,4774,view,0
9,2021-10-21 19:50:09,118420,2450,view,0


In [6]:
data_joined = pd.merge(
                        left = feed_data,
                        right = user_data,
                        left_on="user_id",
                        right_on="user_id"
                       )
data_joined = pd.merge(
    left = data_joined,
    right = post_data,
    left_on="post_id",
    right_on="post_id"
)

In [7]:
data_joined.head(50)


,timestamp,user_id,post_id,action,target,gender,age,country,city,exp_group,os,source,text,topic
0,2021-10-21 19:37:09,118420,1369,view,0,0,41,Russia,Moscow,2,Android,organic,UKIP candidate suspended\n\nEurosceptic party ...,politics
1,2021-10-21 19:37:59,118420,1093,view,0,0,41,Russia,Moscow,2,Android,organic,Hitler row over Welsh arts cash\n\nAn artist c...,politics
2,2021-10-21 19:38:32,118420,2584,view,0,0,41,Russia,Moscow,2,Android,organic,Im thinking that many of the people who discou...,covid
3,2021-10-21 19:39:46,118420,5355,view,0,0,41,Russia,Moscow,2,Android,organic,I actually liked certain things about this gam...,movie
4,2021-10-21 19:42:16,118420,838,view,0,0,41,Russia,Moscow,2,Android,organic,Vera Drake leads UK Oscar hopes\n\nMike Leighs...,entertainment
5,2021-10-21 19:42:31,118420,948,view,0,0,41,Russia,Moscow,2,Android,organic,Blair moves to woo Jewish voters\n\nTony Blair...,politics
6,2021-10-21 19:43:20,118420,6927,view,0,0,41,Russia,Moscow,2,Android,organic,I just wanna say that amongst all the so-calle...,movie
7,2021-10-21 19:45:40,118420,1888,view,0,0,41,Russia,Moscow,2,Android,organic,Parry relishes Anfield challenge\n\nBBC Sport ...,sport
8,2021-10-21 19:48:34,118420,4774,view,0,0,41,Russia,Moscow,2,Android,organic,Lorna Green(Janine Reynaud)is a performance ar...,movie
9,2021-10-21 19:50:09,118420,2450,view,0,0,41,Russia,Moscow,2,Android,organic,Would you take a #Covid19 vaccine? Anti-vaxxer...,covid


In [8]:
categ_cols = data_joined.select_dtypes(include='object')
numeric_cols = data_joined.select_dtypes(exclude='object')

In [9]:
numeric_cols.describe()

,timestamp,user_id,post_id,target,gender,age,exp_group
count,10000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,2021-11-14 13:27:22.027100160,117579.234800,3400.752700,0.105000,0.541300,23.660800,2.070900
min,2021-10-01 13:21:06,4844.000000,3.000000,0.000000,0.000000,14.000000,0.000000
25%,2021-10-22 16:15:10.249999872,118425.000000,1500.000000,0.000000,0.000000,19.000000,1.000000
50%,2021-11-14 14:00:53.500000,118431.000000,3193.000000,0.000000,1.000000,22.000000,2.000000
75%,2021-12-06 06:27:36.500000,118437.000000,5246.000000,0.000000,1.000000,25.000000,3.000000
max,2021-12-29 22:28:56,118442.000000,7319.000000,1.000000,1.000000,41.000000,4.000000
std,NaN,9800.402947,2115.895184,0.306569,0.498316,6.207173,1.397093


In [10]:
categ_cols.describe()

,action,country,city,os,source,text,topic
count,10000,10000,10000,10000,10000,10000,10000
unique,2,3,20,2,2,4640,7
top,view,Russia,Moscow,Android,organic,Fox attacks Blairs Tory lies\n\nTony Blair lie...,movie
freq,8950,9774,1323,6319,9925,14,3743


In [11]:
data_joined["month"] = data_joined["timestamp"].dt.month
data_joined["weekday"] = data_joined["timestamp"].dt.weekday
data_joined["hour"] = data_joined["timestamp"].dt.hour

In [12]:
data_joined = data_joined.drop('timestamp', axis=1)

In [13]:
data_copy = data_joined.copy()

In [14]:
data_copy.describe()

,user_id,post_id,target,gender,age,exp_group,month,weekday,hour
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000
mean,117579.234800,3400.752700,0.105000,0.541300,23.660800,2.070900,10.96370,3.020000,14.313100
std,9800.402947,2115.895184,0.306569,0.498316,6.207173,1.397093,0.82792,1.979493,4.999457
min,4844.000000,3.000000,0.000000,0.000000,14.000000,0.000000,10.00000,0.000000,6.000000
25%,118425.000000,1500.000000,0.000000,0.000000,19.000000,1.000000,10.00000,1.000000,10.000000
50%,118431.000000,3193.000000,0.000000,1.000000,22.000000,2.000000,11.00000,3.000000,14.000000
75%,118437.000000,5246.000000,0.000000,1.000000,25.000000,3.000000,12.00000,5.000000,19.000000
max,118442.000000,7319.000000,1.000000,1.000000,41.000000,4.000000,12.00000,6.000000,23.000000


In [17]:
catboost_model = CatBoostClassifier()